In [57]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

In [58]:
def calc_Pd_Pfa_simple(m, n, p, Nevents, Detection, Stx, SNR, Channel):
    #-----------------------------------------------------------------------#
    # Function that calculates Pd and Pfa
    #-----------------------------------------------------------------------#

    T_H0 = np.zeros(Nevents, dtype=complex)
    T_H1 = np.zeros(Nevents, dtype=complex)

    for ind in range(1, 2 * Nevents + 1):
        #%%%%%%%%%%%%%%%%%%%%%%%
        #%%% Impulsive Noise %%%
        #%%%%%%%%%%%%%%%%%%%%%%%

        #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        #%%%%%% Generate the Channel Array %%%%%%
        #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        H = Gen_Channel_H(m, p, Channel)

        #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        #%%%% Generate the Transmitted Symbols Array %%%%%%
        #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        X = Gen_Tx_Signal_X(p, n, Stx)
        X = X * np.sqrt(10 ** (SNR / 10))

        #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        #%%%%% Generate the Array of Thermal Noise %%%%%%%%
        #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        V = (np.sqrt(1 / 2) * np.random.randn(m, n)) + 1j * (np.sqrt(1 / 2) * np.random.randn(m, n))

        #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        #%%%%% Received sample matrix with a random variable inserted, sometimes transmit, sometimes not transmit broadcasts %%%%%%
        #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        I_tx = 1 if ind > Nevents else 0

        #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        #%%% It takes into account the probability of occurrence of impulsive noise %%%%
        #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        Y = I_tx * np.dot(H, X) + V

        g = np.ones(m)

        #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        #%%%%% Choose the Detection Technique %%%%%%
        #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        T = Gen_Var_T(Detection, m, n, Y, g, SNR)

        #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        #%%%%% For generate the histogram H0 and H1 %%%%%%%
        #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        if I_tx == 1:
            T_H1[ind - Nevents - 1] = abs(T)
        else:
            T_H0[ind - 1] = abs(T)

    return T_H0, T_H1

def Gen_Channel_H(m, p, Channel):
    # This is a placeholder function, replace it with the actual implementation.
    return np.random.randn(m, p) + 1j * np.random.randn(m, p)

def Gen_Tx_Signal_X(p, n, Stx):
    # This is a placeholder function, replace it with the actual implementation.
    return np.random.randn(p, n) + 1j * np.random.randn(p, n)

def Gen_Var_T(Detection, m, n, Y, g, SNR):
    # This is a placeholder function, replace it with the actual implementation based on the Detection technique.
    return np.sum(Y)

In [59]:
def gen_channel_h(m, p, channel):
    """
    Generates the Channel array H (m x p) with Rayleigh normalized samples.

    Parameters:
        m (int): Number of cognitive radios (CR).
        p (int): Number of primary users.
        channel (int): Channel type (1 for Rayleigh, else for deterministic channel).

    Returns:
        H (ndarray): Channel array (m x p).
    """
    if channel == 1:
        H = (np.sqrt(2) / 2) * (np.random.randn(m, p) + 1j * np.random.randn(m, p))
    else:
        H = np.ones((m, p))

    # Uncomment the lines below to normalize H, if required
    # PH = 1 / (m * p) * np.sum(np.diag(np.conj(H.T) @ H))
    # H = H / np.sqrt(PH)

    return H

In [60]:
def gen_tx_signal(p, n, Stx, vec_simbol=None):
    """
    Generates a transmission matrix depending on the chosen signal.

    Parameters:
    p: int - Number of rows of transmission matrix.
    n: int - Number of columns of transmission matrix.
    Stx: int - Signal type (1: Noise, 2: BPSK, 3: MQAM, 4: Customized).
    vec_simbol: numpy.ndarray (optional) - Vector containing symbols for MQAM or customized signal.

    Returns:
    numpy.ndarray - Transmission matrix X.
    """
    if Stx == 1:
        # Noise-like signal
        X = (np.sqrt(2) / 2) * (np.random.randn(p, n) + 1j * np.random.randn(p, n))

    elif Stx == 2:
        # BPSK Signal
        X = 2 * (np.random.randint(0, 2, (p, n)) - 0.5)

    elif Stx == 3:
        # MQAM Signal
        if vec_simbol is None:
            raise ValueError("For MQAM, vec_simbol must be provided.")

        # Calculate energy and normalize the symbol vector
        energy = (np.sum(np.real(vec_simbol)**2) + np.sum(np.imag(vec_simbol)**2)) / len(vec_simbol)
        vec_simbol = vec_simbol / np.sqrt(energy)

        # Generate transmission matrix X
        X = vec_simbol[np.random.randint(0, len(vec_simbol), (p, n))]

    elif Stx == 4:
        # Customized Signal (OFDM - ISDB-TB)
        fs = 512 / 63 * 10**6  # Sampling frequency
        modo = 3
        IG = 1 / 32  # Guard interval
        M = 64  # Modulation order
        Rcod = 7 / 8  # Coding rate
        SNRdB = 50  # SNR in dB
        Cmp = 1  # Channel multiplier

        # Determine Mult_modo based on operation mode
        if modo == 1:
            mult_modo = 1
        elif modo == 2:
            mult_modo = 2
        else:
            mult_modo = 4

        # Number of active carriers
        nc_ativas = 13 * 108 * mult_modo + 1
        nc_util = 13 * 96 * mult_modo
        nc = 2048 * mult_modo  # Total number of carriers

        # Generate parallel symbols with useful data and pilots
        simb_tx_paralelo = np.random.randint(0, M, nc_ativas)  # Random symbols
        dist_pilot = 12
        pos_pilotos = np.arange(0, len(simb_tx_paralelo), dist_pilot)
        pilotos = np.zeros(nc_ativas)
        pilotos[::dist_pilot] = (np.random.randint(0, 2, len(pos_pilotos)) - 0.5) * 2 * 4 / 3
        simb_tx_paralelo[::dist_pilot] = pilotos[::dist_pilot]

        # Add null carriers
        simb_tx_completo = np.zeros(nc, dtype=complex)
        simb_tx_completo[:len(simb_tx_paralelo) // 2 + 1] = simb_tx_paralelo[:len(simb_tx_paralelo) // 2 + 1]
        simb_tx_completo[-len(simb_tx_paralelo) // 2:] = simb_tx_paralelo[len(simb_tx_paralelo) // 2 + 1:]

        # Generate OFDM symbol
        simb_ofdm = np.fft.ifft(simb_tx_completo, nc)

        # Generate guard interval
        simb_ofdm_tg = np.concatenate((simb_ofdm[-int(len(simb_ofdm) * IG):], simb_ofdm))

        # Normalize vector symbols
        energy = (np.sum(np.real(simb_ofdm_tg)**2) + np.sum(np.imag(simb_ofdm_tg)**2)) / len(simb_ofdm_tg)
        vec_simbol = simb_ofdm_tg / np.sqrt(energy)

        # Generate transmission matrix X
        X = vec_simbol[:n]

    else:
        raise ValueError("Invalid Stx value. Must be 1, 2, 3, or 4.")

    return X

In [61]:
def gen_var_t(detection, m, n, y, g, snr):
    """
    Generate random decision variable T based on the technique chosen by the user.

    Parameters:
    detection: int - Technique chosen by the user (1: ED, 2: ERD-MMED, 3: RLRT, 4: GLRT, 5: Customized User Technique)
    m: int - Number of cognitive radios
    n: int - Number of samples collected by each CR
    y: numpy array (mxn) - Array of samples
    g: numpy array - Gain used to simulate the LNA and AGC
    snr: float - Signal to Noise Ratio

    Returns:
    T: float - Decision variable T based on the technique chosen
    """
    # Covariance Matrix
    W = np.matmul(y, np.conjugate(y.T)) / n

    # Eigenvalues of W
    lambda_vals = np.sort(np.linalg.eigvals(W))[::-1]

    # Initialize T
    T = None

    if detection == 1:
        # Energy Detection (ED)
        T = np.sum(np.abs(y)**3)

    elif detection == 2:
        # Eigenvalue Ratio Detector - Maximum to Minimum Eigenvalue (ERD-MMED)
        T = lambda_vals[0] / lambda_vals[m - 1]

    elif detection == 3:
        # Rao's Likelihood Ratio Test (RLRT)
        soma = np.sum(g**2 * (10**(snr / 10))**-1)
        Tden = soma / m
        T = lambda_vals[0] / Tden

    elif detection == 4:
        # Generalized Likelihood Ratio Test (GLRT)
        T = lambda_vals[0] / (np.sum(np.diag(W)) / m)

    elif detection == 5:
        # Customized User Technique
        # Implement custom logic here
        T = None  # Placeholder for user-defined technique

    else:
        raise ValueError("Invalid detection technique chosen")

    return T

In [62]:
def gera_simb(M, N):
    # This function generates N random symbols of M-QAM

    simbolos = []
    for t in range(-(int(np.sqrt(M)) - 1), int(np.sqrt(M)), 2):
        for q in range(-(int(np.sqrt(M)) - 1), int(np.sqrt(M)), 2):
            simbolos.append(t + 1j * q)

    S1 = np.random.choice(simbolos, N)

    # Energy Normalization
    if M == 4:
        S1 = S1 / np.sqrt(2)
        cte_norm = np.sqrt(2)
    elif M == 16:
        S1 = S1 / np.sqrt(10)
        cte_norm = np.sqrt(10)
    elif M == 64:
        S1 = S1 / np.sqrt(42)
        cte_norm = np.sqrt(42)
    else:
        cte_norm = 1  # Default normalization factor if not specified

    return S1, cte_norm

# Example usage
M = 16
N = 10
Simbolos_Mqam, cte_norm = gera_simb(M, N)
print("Generated Symbols:", Simbolos_Mqam)
print("Normalization Constant:", cte_norm)

Generated Symbols: [-0.31622777-0.9486833j  -0.31622777+0.9486833j  -0.31622777-0.31622777j
 -0.31622777-0.9486833j   0.31622777-0.31622777j  0.9486833 +0.31622777j
 -0.9486833 +0.31622777j  0.9486833 -0.9486833j   0.9486833 -0.31622777j
  0.31622777-0.31622777j]
Normalization Constant: 3.1622776601683795


In [ ]:
# Spectral Sensing in TVWS
# Date: 01/2023
# Authors: André dos Anjos and Dayane Cristina
# Description: This script is the main code for Spectral Sensing Simulation

# Input Parameters
m = 1  # Number of cognitive radios
n = 500000  # Number of samples in the vector
p = 1  # Number of primary users
Nevents = 10000  # Number of Monte Carlo events in the simulation: FA or Det
Detection = 1  # Detection: 1-ED, 2-ERD, 3-RLRT, 4-GLRT, 5-Customized
Stx = 4  # Transmitted signal type: 1-Noise or 2-BPSK, 3M-QAM, 4-ISDB-T OFDM
Channel = 1  # Channel: 0-AWGN, 1-Rayleigh
SNR = -10  # Signal-to-noise ratio in dB
N_pts = 100  # Number of ROC curve points

# Main Execution
print('Initializing processing')
H_0, H_1 = calc_Pd_Pfa_simple(m, n, p, Nevents, Detection, Stx, SNR, Channel)
print('Processing completed')

# Generating ROC Curve
T_ED_H0 = H_0
T_ED_H1 = H_1
range_total = max(T_ED_H1) - min(T_ED_H0)

# Finding a valid range of thresholds
threshold = np.linspace(min(T_ED_H0) - 0.05 * range_total, max(T_ED_H1) + 0.05 * range_total, N_pts)

# Calculating detection and false alarm probabilities for a given SNR
Pfa = np.zeros(len(threshold))
Pd = np.zeros(len(threshold))
for i in range(len(threshold)):
    Pfa[i] = np.sum(T_ED_H0 > threshold[i]) / len(T_ED_H0)
    Pd[i] = np.sum(T_ED_H1 > threshold[i]) / len(T_ED_H1)

# Calculating measured SNR
SNRonSimulation = 10 * np.log10(np.mean(T_ED_H1) / np.mean(T_ED_H0) - 1)

# Clear the zero value from vector
H0_min = np.min(H_0)
H0_max = np.max(H_0)
H1_min = np.min(H_1)
H1_max = np.max(H_1)

# Build the y value to histogram
amp_H0, x_axes_H0 = np.histogram(H_0, bins=100)
amp_H1, x_axes_H1 = np.histogram(H_1, bins=100)

# Graph 1: P_D and P_FA vs. Threshold
plt.figure(1)
plt.plot(threshold, Pd, '--gs', linewidth=2, label='$P_D$')
plt.plot(threshold, Pfa, '--rs', linewidth=2, label='$P_{FA}$')
plt.title('Graph $P_D$ and $P_{FA}$ vs. Threshold')
plt.xlabel('$\gamma$ (threshold)')
plt.ylabel('$P_D$ and $P_{FA}$')
plt.legend()
plt.grid(True)
plt.axis([min(threshold), max(threshold), 0, 1])
plt.show()

# Graph 2: Normal ROC - P_D vs. P_FA
plt.figure(2)
plt.plot(Pfa, Pd, 'r', linewidth=1)
plt.title('Normal ROC - $P_D$ vs. $P_{FA}$')
plt.xlabel('$P_{FA}$')
plt.ylabel('$P_D$')
plt.grid(True)
plt.axis([0, 1, 0, 1])
plt.show()

# Graph 3: Complementary ROC
plt.figure(3)
plt.loglog(Pfa, (1 - Pd), '--bs', linewidth=2)
plt.title('Complementary ROC')
plt.xlabel('$P_{FA}$')
plt.ylabel('$P_{FN}$')
plt.legend(['$P_{FN}$ vs. $P_{FA}$'])
plt.grid(True)
plt.axis([0, 1, 0, 1])
plt.show()

# Graph 4: Histogram
plt.figure(4)
plt.bar(x_axes_H0[:-1], amp_H0, width=np.diff(x_axes_H0), color='b', alpha=0.5, label='Histogram H0')
plt.bar(x_axes_H1[:-1], amp_H1, width=np.diff(x_axes_H1), color='r', alpha=0.5, label='Histogram H1')
plt.xlabel('Variable T')
plt.ylabel('PDF')
plt.legend()
plt.show()

# System Message
print('Process Completed')

Initializing processing
